### 기본 세팅

In [2]:
import os
import pandas as pd
import random
import re

from tqdm.notebook import tqdm
from kss import split_sentences

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import csv

### 상담 텍스트 수집

In [4]:
def get_상담_info():
    
    data_path = './상담_데이터'     # 데이터 저장 경로
    data_list = os.listdir(data_path)
    total_text_list  = []           # 개별 상담 내용
    total_text_count = []           # 개별 상담 문장 개수
    
    
    # 폴더에 접근해서 텍스트 수집
    for folder in tqdm(data_list):
        # 개별 상담 건수에 접근
        file_path = data_path + '/' + folder + '/'
        file_list = os.listdir(file_path)
        file_list_py = [file for file in file_list if file.endswith('.txt')] 


        # 문장별로 나뉜 텍스트를 합침
        total_txt = ''
        for file_name in file_list_py:
            text_file = open(file_path+file_name, 'r', encoding='UTF8')
            line = text_file.readline()
            total_txt += line
            
        total_text_list.append(total_txt)
        total_text_count.append(len(file_list_py))
    
    
    counseling_전체 = pd.DataFrame({'인덱스': data_list, '내용': total_text_list, '문장 개수': total_text_count})
    return counseling_전체

In [6]:
counseling_전체 = get_상담_info()
#counseling_전체 = pd.read_csv('counseling_전체.csv', index_col=0)

  0%|          | 0/8319 [00:00<?, ?it/s]

In [7]:
counseling_전체

,인덱스,내용,문장 개수
0,S00008000,n/ 안녕하세요 고객님. 크다 증권 이정연입니다.n/ 수고 많으십니다. 다른 게 아...,47
1,S00008001,"n/ 쉐어링 카드 상담원 김지은입니다. 무엇을 도와드릴까요?n/ 네, 이번에 하는 ...",39
2,S00008002,"n/ 안녕하십니까. 겨울 카드사 상담사 이여름입니다.n/ 네, 안녕하세요. 카드사에...",32
3,S00008003,n/ 안녕하세요. 쉐어링 카드 김예닮 상담원입니다.여보세요? 제 목소리 들리세요?네...,29
4,S00008004,n/ 여보세요?n/ 안녕하세요. 저/ 며칠 전에 전화드렸었는데 기억하실지 모르겠네요...,18
...,...,...,...
8314,S00016766,n/ 안녕하세요. 제가 이번에 차를 한대 새로 뽑을 예정이어서 자동차 할부를 좀 알...,24
8315,S00016767,안녕하세요. 고생하시네요. 제가 주부인데요.저 급하게 돈이 필요한데 제가 주부라 소...,23
8316,S00016769,n/ 고생하십니다. 요즘 은행 금리가 완전 바닥이잖아요.n/ 그래서 적금을 가입해야...,25
8317,S00016770,고생하십니다. 제가 문자 (1)/(하나)를 받았는데요. 요즘 스팸이 하도 많아서 발...,24


### 다운 샘플링

In [8]:
# 데이터 정보
counseling_전체_text = list(counseling_전체['내용'])
len_상담_text = len(counseling_전체_text)
print(f' 다운 샘플링 전 상담 데이터 건수: {len_상담_text}')

counseling_전체_count = list(counseling_전체['문장 개수'])
sum_상담_count = sum(counseling_전체_count)
print(f' 문장 분리 전 상담 데이터 문장 개수: {sum_상담_count}')

 다운 샘플링 전 상담 데이터 건수: 8319
 문장 분리 전 상담 데이터 문장 개수: 245066


In [ ]:
# 문장 분리
for i in tqdm(range(len_상담_text)):
    cur_text = counseling_전체_text[i]
    splited_text = []
        
    # 문장 분리 후 길이 저장
    splited = split_sentences(cur_text)
    len_splited = len(splited)

    # 문장을 나눠서 추가함
    for j in range(len_splited):
        splited_text.append(splited[j])
    
    # 분리한 문장으로 대체
    counseling_전체_text[i] = splited_text

  0%|          | 0/8319 [00:00<?, ?it/s]

[Korean Sentence Splitter]: Initializing Pynori...


In [10]:
# 수정된 문장 개수 반영
for i in range(len_상담_text):
    cur_text = counseling_전체_text[i]
    len_cur_text = len(cur_text)
    counseling_전체_count[i] = len_cur_text
    
sum_상담_count = sum(counseling_전체_count)
print(f' 문장 분리 후 상담 데이터 문장 개수: {sum_상담_count}')
    
counseling_전체['내용'] = counseling_전체_text
counseling_전체['문장 개수'] = counseling_전체_count

 문장 분리 후 상담 데이터 문장 개수: 415426


In [8]:
def down_sampling(dataframe):
    
    # 샘플링 성공 플래그 변수
    success = False
    
    
    # 보이스피싱 건수와 맞게 다운 샘플링
    while(not success):
        before = dataframe
        randnum = random.randrange(1, 1501)
        
        # 건수를 200개로 다운 샘플링
        down_sampled = before.sample(n=200, random_state=randnum).reset_index(drop=True)
        down_sampled_count = list(down_sampled['문장 개수'])
        print(f' 다운 샘플링 된 전체 문장 개수: {sum(down_sampled_count)}')
        
        # 보이스피싱 건수의 오차범위 +/- 1%에 들면 성공
        if (10793*0.99 <= sum(down_sampled_count)) and (sum(down_sampled_count) <= 10793*1.01):
            print(' 다운 샘플링 성공')
            success = True

    
    return down_sampled

In [9]:
# 다운 샘플링 진행
counseling_전체 = down_sampling(counseling_전체)

 다운 샘플링 된 전체 문장 개수: 10725
 다운 샘플링 성공


In [10]:
counseling_전체_text = list(counseling_전체['내용'])
len_상담_text = len(counseling_전체_text)
print(f' 다운 샘플링 후 상담 데이터 건수: {len_상담_text}')

 다운 샘플링 후 상담 데이터 건수: 200


### 텍스트 전처리

In [11]:
# 정규표현식 적용
for i in range(len_상담_text):
    cur_text = counseling_전체_text[i]
    
    for j in range(len(cur_text)):
        # 개행문자 제거
        cur_sentence = cur_text[j]
        cur_sentence = re.sub(r'n/ ', '', cur_sentence)


        # + 표현 제거
        cnt = 0
        matchs0 = re.finditer(r'\+', cur_sentence)
        for match in matchs0:
            cnt += 1

        while cnt > 0:
            matchs1 = re.finditer(r'\+', cur_sentence)
            for match in matchs1:
                plus_index = match.span()[0]
                break
            plus_index_saved = plus_index
            
            blank_index = -1
            while plus_index >= 0:
                if cur_sentence[plus_index] == ' ':
                    blank_index = plus_index
                    break
                else:
                    plus_index -= 1
            
            if blank_index == -1: blank_index = 0
            dropword = cur_sentence[blank_index:plus_index_saved+1]
            cur_sentence = cur_sentence.replace(dropword, '')
            cnt -= 1


        # ()/() 표현 제거
        cnt = 0
        matchs0 = re.finditer(r'\([^)]*\)/\([^)]*\)', cur_sentence)
        for match in matchs0:
            cnt += 1

        while cnt > 0:
            matchs1 = re.finditer(r'\([^)]*\)/\([^)]*\)', cur_sentence)
            for match in matchs1:
                start_idx = match.span()[0]
                break

            matchs2 = re.finditer(r'\)\/\(', cur_sentence)
            for match in matchs2:    
                center_idx = (match.span()[0] + match.span()[1]) // 2
                break

            dropword = cur_sentence[start_idx:center_idx+1]
            cur_sentence = cur_sentence.replace(dropword, '')
            cnt -= 1

        cur_sentence = re.sub(r'/', '', cur_sentence)
        cur_sentence = re.sub(r'\(', '', cur_sentence)
        cur_sentence = re.sub(r'\)', '', cur_sentence)


        # 정제된 문장으로 대체 
        cur_text[j] = cur_sentence
        counseling_전체_text[i] = cur_text

### 파일 저장

In [12]:
counseling_전체 = counseling_전체.drop(['인덱스', '문장 개수'], axis='columns')
counseling_전체.head(10)

,내용
0,"[네, 안녕하세요., 오늘 제가 연락드린 건 제가 지금 대출을 받아서 갚고 있는 중..."
1,"[문의하고 싶은 게 있어서요., 담보대출 관련 문의 좀 드릴게요., 죄송한데 제가 ..."
2,"[안녕하세요., 국민은행 상담사 김지원입니다., 네, 안녕하세요., 무엇을 도와드릴..."
3,"[안녕하세요., 미스 은행 상담원 은가은입니다., 무엇을 도와드릴까요?, 네, 안녕..."
4,"[네 안녕하세요., 네 제가 선물옵션 투자를 좀 하고 싶어서 전화드렸습니다., 제가..."
5,"[네, 결혼 자금으로 적금 든 게 있는데, 집 계약 문제로 돈이 필요해서 적금을 해..."
6,"[아 네, 안녕하세요., 아이 앞으로 예금하고 적금을 들고 싶은데요?, 은행에 직접..."
7,"[네, 안녕하세요,, 몇 가지 여쭐 게 있어서 연락드렸어요., 아니요,, 저 회원은..."
8,"[안녕하세요., 쉐어링 카드 상담원 홍길동입니다., 네, 안녕하세요., 전화로 카드..."
9,"[안녕하세요., 네, 안녕하세요., 저희 할머니가 좀 전에 이상한 전화를 받았는데 ..."


In [13]:
# 파일 저장
counseling_전체.to_csv('counseling_전체.csv')